In [ ]:
from cylinderConvection.cylinderConvection import cylConvection
import numpy as np

## Quick example

Create a class instance each time you set Γ, M, K, J:

In [ ]:
CylConvection = cylConvection(Γ = 4, M = 30,K = 30, J = 25)

Compute the minimum Rayleigh numbers for oscillatory and wall convection, given Ekman and Prandtl numbers. Print values.

In [ ]:
CylConvection.minimizeRayleigh(Ek = 1e-4, Pr = 0.025, printVals=True)

## Looping over a range of Ekman numbers

Create a class instance each time you set Γ, M, K, J (should take ~50 seconds):

In [ ]:
CylConvection = cylConvection(Γ = 2, M = 90,K = 90, J = 100)

Compute the minimum Rayleigh number for each Ekman number in an array of values (should take ~2 seconds):

In [ ]:
Pr = 0.025
numE = 100;
EkArr = np.logspace(-8,-3,num=numE)
RaOCylArr = np.zeros(len(EkArr))
RaWArr = np.zeros(len(EkArr))
m0cArr = np.zeros(len(EkArr))
k0cArr = np.zeros(len(EkArr))

for i in range(len(EkArr)):
    Ek = EkArr[i]
    R1c, σ1c, m0c, k0c, Rwall, σwall, mwall = CylConvection.minimizeRayleigh(Ek = Ek,Pr = Pr)
    RaOCylArr[i] = R1c/Ek
    RaWArr[i] = Rwall/Ek
    m0cArr[i] = m0c
    k0cArr[i] = k0c

Plot:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

colors = ['#4E8FC8','#D9A900','#C50230']

E_filt = EkArr[np.argmax(np.abs(np.diff(k0cArr)))+1:-15]
k0c_filt = k0cArr[np.argmax(np.abs(np.diff(k0cArr)))+1:-15]
m,b = np.polyfit(np.log(E_filt),np.log(k0c_filt),1)

f, (ax1, ax2) = plt.subplots(2, 1,figsize=(7,7))
title = f'$\\Gamma = {CylConvection.Γ:.2f},\\quad Pr = {Pr:.3f},'
title += f'\\quad M={CylConvection.M:.0f}, \\quad K={CylConvection.K:.0f}, \\quad J={CylConvection.J:.0f}$'
ax1.set_title(title)
ax1.loglog(EkArr,Rwall/EkArr,'k--',label='$Ra_W$')
ax1.loglog(EkArr,RaOCylArr,color=colors[1],linestyle='--',label='$Ra_O^{cyl}$')
ax1.invert_xaxis()
ax1.set_xlim(EkArr[-1],EkArr[0])
ax1.set_ylim(1e4,1e11)
ax1.grid()
ax1.set_xlabel('$Ek$')
ax1.set_ylabel('$Ra$')
ax1.legend()

ax2.loglog(EkArr,m0cArr,label='$m_{0c}$',color='#915eb5')
ax2.loglog(EkArr,k0cArr,label='$k_{0c}$',color='#e39600')
ax2.loglog(EkArr[EkArr<=E_filt[-1]],np.exp(np.log(EkArr[EkArr<=E_filt[-1]])*m + b),color='gray',linestyle='--',label=f'$k$ = {np.exp(b):.2f} $Ek^{{{m:.2f}}}$')
ax2.invert_xaxis()
ax2.set_yticks(np.array([1,5,10,50,100]),labels=['1','5','10','50','100'])
ax2.grid(color='#D3D3D3')
ax2.set_xlim(EkArr[-1],EkArr[0])
ax2.set_xlabel('$Ek$')
ax2.set_ylabel('critical wavenumber (oscillatory)')
ax2.legend(loc='upper left')
plt.show()

## Check against values from [Zhang & Liao (2017)](https://doi.org/10.1017/9781139024853)

In [ ]:
EkZL,PrZL,ΓZL,k0cZL,m0cZL,R1cZL,σ1cZL = np.loadtxt('Zhang&Liao2017Values.csv',skiprows=1,delimiter=',',unpack=True)
ΓZL = np.unique(ΓZL).item()

In [ ]:
CylConvection = cylConvection(Γ = ΓZL, M = 25,K = 25, J = 25)

In [ ]:
R1cArr = np.zeros(len(EkZL))
σ1cArr = np.zeros(len(EkZL))
m0cArr = np.zeros(len(EkZL))
k0cArr = np.zeros(len(EkZL))

for i in range(len(EkZL)):
    Ek = EkZL[i]
    Pr = PrZL[i]
    R1c, σ1c, m0c, k0c, Rwall, σwall, mwall = CylConvection.minimizeRayleigh(Ek = Ek,Pr = Pr)
    R1cArr[i] = R1c
    σ1cArr[i] = σ1c
    m0cArr[i] = m0c
    k0cArr[i] = k0c

numWrongR1c = np.sum(np.abs((R1cZL - R1cArr)/R1cZL) > 1e-3)
numWrongσ1c = np.sum(np.abs((σ1cZL - σ1cArr)/σ1cZL) > 1e-3)
numWrongm0c = np.sum(np.abs(m0cArr-m0cZL))
numWrongk0c = np.sum(np.abs(k0cArr-k0cZL))

if (numWrongR1c+numWrongσ1c+numWrongm0c+numWrongk0c) == 0:
    msg = f'Computed values are within 0.1% of published values (at'
    msg+= f' M = {CylConvection.M:.0f}, K = {CylConvection.K:.0f}, and J = {CylConvection.J:.0f})'
    print(msg)
else:
    print("Something's wrong.")